In [1]:
import sys
sys.path.append('/home/s94zalek/shape_matching')

# set logging level to info
import logging
logging.basicConfig(level=logging.INFO)

from datasets_code import build_dataloader, build_dataset
from utils.options import parse_options
from train import create_train_val_dataloader
import torch
import numpy as np

import os
os.chdir('/home/s94zalek/shape_matching')

# print current working directory
print(os.getcwd())

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
/home/s94zalek/shape_matching


In [2]:
import os

# root_path = osp.abspath(osp.join(__file__, osp.pardir))
root_path = '/home/s94zalek/shape_matching'

opt = parse_options(root_path, is_train=False, use_argparse=False,
                    opt_path = 'options/train/faust.yaml')

opt['root_path'] = root_path
opt['dist'] = False

opt['datasets']['train_dataset']['return_corr'] = True
opt['datasets']['train_dataset']['return_dist'] = False
opt['datasets']['test_dataset']['return_dist'] = False

# create train and validation dataloaders
result = create_train_val_dataloader(opt)
train_loader, train_sampler, val_loader, total_epochs, total_iters = result

test_set = build_dataset(opt['datasets']['test_dataset'])
test_loader = build_dataloader(
test_set, opt['datasets']['test_dataset'], phase='val', num_gpu=opt['num_gpu'], dist=opt['dist'], sampler=None, seed=opt['manual_seed'])


Backend DataParallel.
Path already exists. Rename it to /home/s94zalek/shape_matching/results/faust_archived_20240502_123311


Calculating functional maps: 100%|█████████████████████████████████████████████████████████████████████████████████| 6400/6400 [01:10<00:00, 90.83it/s]
2024-05-02 12:34:26,178 INFO: Dataset [PairFaustDataset]-[FaustTrain] is built.
Calculating functional maps: 100%|██████████████████████████████████████████████████████████████████████████████████| 400/400 [00:03<00:00, 102.82it/s]
2024-05-02 12:34:30,854 INFO: Dataset [PairFaustDataset]-[FaustTest] is built.
Calculating functional maps: 100%|██████████████████████████████████████████████████████████████████████████████████| 400/400 [00:03<00:00, 103.35it/s]
2024-05-02 12:34:35,380 INFO: Dataset [PairFaustDataset]-[FaustTest] is built.


In [12]:
Cxy_40 = []
Dx_40 = []
Dy_40 = []

Vxy_computed_40 = []
Rxy_computed_40 = []

data_40 = []

train_dataset = train_loader.dataset
for i in range(len(train_dataset)):
    data = train_dataset[i]
    if data['first']['name'] == 'tr_reg_040':
        
        Cxy_40.append(data['Cxy'])
        
        Dx_40.append(data['first']['evals'])
        Dy_40.append(data['second']['evals'])
        
        Vxy_computed_40.append(data['Vxy'])
        Rxy_computed_40.append(data['Rxy'])
        
        data_40.append(data)
        
Cxy_40_full = torch.stack(Cxy_40)
Cxy_40_truncated = torch.stack(Cxy_40)[:, :20, :20]

Dx_40_full = torch.stack(Dx_40)
Dy_40_full = torch.stack(Dy_40)

Vxy_computed_40_full = torch.stack(Vxy_computed_40)
Rxy_computed_40_full = torch.stack(Rxy_computed_40)

In [13]:
########################################################################################
# Check if difference operators in dataset are correct
########################################################################################

In [5]:
Cxy_T_Cxy_40_full = torch.bmm(Cxy_40_full.transpose(1, 2), Cxy_40_full)

Rxy_40_left = torch.bmm(
    torch.diag_embed(-1 / Dx_40_full),
    Cxy_40_full.transpose(1, 2)
    )
Rxy_40_right = torch.bmm(
    torch.diag_embed(-Dy_40_full),
    Cxy_40_full
    )

Rxy_40_full = torch.bmm(Rxy_40_left, Rxy_40_right)

In [15]:
print('Vxy in dataset - Vxy computed:', (Vxy_computed_40_full - Cxy_T_Cxy_40_full).sum())
print('Rxy in dataset - Rxy computed:', (Rxy_40_full - Rxy_computed_40_full).sum())

Vxy in dataset - Vxy computed: tensor(0.0002)
Rxy in dataset - Rxy computed: tensor(-6.4227e-05)


In [16]:
########################################################################################
#
# Descriptor preservation by GT Fmap
#
########################################################################################

In [41]:
data = data_40[21]
data_x = data['first']
data_y = data['second']

data['Cxy'].shape, data_x['evecs_trans'].shape, data_y['evecs'].shape

(torch.Size([200, 200]), torch.Size([200, 5001]), torch.Size([5001, 200]))

In [46]:
# indicator functions as descriptors

ind_pres_loss = (data['Cxy'] @ data_x['evecs_trans'] - data_y['evecs_trans']).abs()

print('Indicator preserv. loss,  50 ef, sum:',
      ind_pres_loss[:50, :50].sum(), 'mean:', ind_pres_loss[:50, :50].mean())
print('Indicator preserv. loss, 100 ef, sum:', ind_pres_loss[:100, :100].sum(),
      'mean:', ind_pres_loss[:100, :100].mean())
print('Indicator preserv. loss, 150 ef, sum:', ind_pres_loss[:150, :150].sum(),
      'mean:', ind_pres_loss[:150, :150].mean())
print('Indicator preserv. loss, 200 ef, sum:', ind_pres_loss[:200, :200].sum(),
      'mean:', ind_pres_loss[:200, :200].mean())

Indicator preserv. loss,  50 ef, sum: tensor(0.3631) mean: tensor(0.0001)
Indicator preserv. loss, 100 ef, sum: tensor(1.6721) mean: tensor(0.0002)
Indicator preserv. loss, 150 ef, sum: tensor(3.8614) mean: tensor(0.0002)
Indicator preserv. loss, 200 ef, sum: tensor(5.6738) mean: tensor(0.0001)


In [ ]:
# I guess this counts as preservation?

In [47]:
import utils.geometry_util as geometry_util

# hks as descriptors
hks_x = geometry_util.compute_hks_autoscale(data_x['evals'].unsqueeze(0), data_x['evecs'].unsqueeze(0), 16)[0]
hks_y = geometry_util.compute_hks_autoscale(data_y['evals'].unsqueeze(0), data_y['evecs'].unsqueeze(0), 16)[0]

hks_pres_loss = (data['Cxy'] @ data_x['evecs_trans'] @ hks_x - data_y['evecs_trans'] @ hks_y).abs()

print('HKS preserv. loss,  50 ef, sum:', hks_pres_loss[:50, :50].sum(), 'mean:', hks_pres_loss[:50, :50].mean())
print('HKS preserv. loss, 100 ef, sum:', hks_pres_loss[:100, :100].sum(), 'mean:', hks_pres_loss[:100, :100].mean())
print('HKS preserv. loss, 150 ef, sum:', hks_pres_loss[:150, :150].sum(), 'mean:', hks_pres_loss[:150, :150].mean())
print('HKS preserv. loss, 200 ef, sum:', hks_pres_loss[:200, :200].sum(), 'mean:', hks_pres_loss[:200, :200].mean())

HKS preserv. loss,  50 ef, sum: tensor(17.4320) mean: tensor(0.0218)
HKS preserv. loss, 100 ef, sum: tensor(21.4995) mean: tensor(0.0134)
HKS preserv. loss, 150 ef, sum: tensor(24.1054) mean: tensor(0.0100)
HKS preserv. loss, 200 ef, sum: tensor(25.4915) mean: tensor(0.0080)


In [50]:
# wks as descriptors

wks_x = geometry_util.compute_wks_autoscale(data_x['evals'].unsqueeze(0), data_x['evecs'].unsqueeze(0), data_x['mass'].unsqueeze(0))[0]
wks_y = geometry_util.compute_wks_autoscale(data_y['evals'].unsqueeze(0), data_y['evecs'].unsqueeze(0), data_y['mass'].unsqueeze(0))[0]

wks_pres_loss = (data['Cxy'] @ data_x['evecs_trans'] @ wks_x - data_y['evecs_trans'] @ wks_y).abs()

print('WKS preserv. loss,  50 ef, sum:', wks_pres_loss[:50, :50].sum(), 'mean:', wks_pres_loss[:50, :50].mean())
print('WKS preserv. loss, 100 ef, sum:', wks_pres_loss[:100, :100].sum(), 'mean:', wks_pres_loss[:100, :100].mean())
print('WKS preserv. loss, 150 ef, sum:', wks_pres_loss[:150, :150].sum(), 'mean:', wks_pres_loss[:150, :150].mean())
print('WKS preserv. loss, 200 ef, sum:', wks_pres_loss[:200, :200].sum(), 'mean:', wks_pres_loss[:200, :200].mean())

WKS preserv. loss,  50 ef, sum: tensor(25.3663) mean: tensor(0.0101)
WKS preserv. loss, 100 ef, sum: tensor(77.3188) mean: tensor(0.0077)
WKS preserv. loss, 150 ef, sum: tensor(126.9926) mean: tensor(0.0066)
WKS preserv. loss, 200 ef, sum: tensor(149.6696) mean: tensor(0.0058)


In [ ]:
########################################################################################
#
# Does this count as descriptor preservation?
#
########################################################################################